# Predict NER Model

In [45]:
requirements = [["spacy", "3.0.6"], ["boto3", "1.9.205"]]

In [46]:
import spacy
import os
import boto3
import json
# from dotenv import load_dotenv, find_dotenv #while running locally

In [47]:
# load_dotenv(find_dotenv())

In [48]:
my_bucket = "DH-SECURE-OCHATTER"
conn = boto3.client(service_name='s3',
        aws_access_key_id= os.environ.get("s3-access-key"),
        aws_secret_access_key= os.environ.get("s3-secret-key"),
        endpoint_url='https://s3.upshift.redhat.com/')

In [49]:
# following commented code is used for saving objects to the bucket and interacting with bucket
# not neccesary in actual workflow

In [50]:
# for key in conn.list_objects(Bucket=my_bucket)['Contents']:
#     print(key['Key'])

In [51]:
# nlp = spacy.load("en_core_web_sm")

In [52]:
# config = nlp.config
# bytes_data = nlp.to_bytes()

In [53]:
# with open('config.json', 'w') as fp:
#     json.dump(config, fp)

# b = open('model_in_bytes', 'wb')
# b.write(bytes_data)
# b.close()

In [54]:
# conn.upload_file(Filename='model_in_bytes',Bucket=my_bucket, Key='model_deploy/model_in_bytes')
# conn.upload_file(Filename='config.json',Bucket=my_bucket, Key='model_deploy/config.json')

In [55]:
obj1 = conn.get_object(Bucket=my_bucket, Key = 'model_deploy/model_in_bytes')

In [56]:
obj1

{'ResponseMetadata': {'RequestId': 'tx00000000000000e42ef01-0060c12d60-65334973-default',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-length': '15831120',
   'accept-ranges': 'bytes',
   'last-modified': 'Wed, 09 Jun 2021 19:22:31 GMT',
   'x-rgw-object-type': 'Normal',
   'etag': '"1a191bd2ae70da9ba51204b2b31d7649-2"',
   'x-amz-request-id': 'tx00000000000000e42ef01-0060c12d60-65334973-default',
   'content-type': 'binary/octet-stream',
   'date': 'Wed, 09 Jun 2021 21:06:40 GMT',
   'connection': 'Keep-Alive',
   'strict-transport-security': 'max-age=31536000; includeSubDomains; preload'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2021, 6, 9, 19, 22, 31, tzinfo=tzutc()),
 'ContentLength': 15831120,
 'ETag': '"1a191bd2ae70da9ba51204b2b31d7649-2"',
 'ContentType': 'binary/octet-stream',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x7f57cd557550>}

In [57]:
obj2 = conn.get_object(Bucket=my_bucket, Key = 'model_deploy/config.json')

In [58]:
obj2

{'ResponseMetadata': {'RequestId': 'tx000000000000019c4934a-0060c12d62-60765d10-default',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-length': '5017',
   'accept-ranges': 'bytes',
   'last-modified': 'Wed, 09 Jun 2021 19:22:31 GMT',
   'x-rgw-object-type': 'Normal',
   'etag': '"fecc39263771ebd53d9bd453ff3da01a"',
   'x-amz-request-id': 'tx000000000000019c4934a-0060c12d62-60765d10-default',
   'content-type': 'binary/octet-stream',
   'date': 'Wed, 09 Jun 2021 21:06:42 GMT',
   'connection': 'Keep-Alive',
   'strict-transport-security': 'max-age=31536000; includeSubDomains; preload'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2021, 6, 9, 19, 22, 31, tzinfo=tzutc()),
 'ContentLength': 5017,
 'ETag': '"fecc39263771ebd53d9bd453ff3da01a"',
 'ContentType': 'binary/octet-stream',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x7f57cd557b70>}

In [59]:
data = obj1['Body'].read()

In [60]:
type(data)

bytes

In [61]:
config_data = obj2['Body'].read()

In [62]:
config_data

b'{"paths": {"train": "corpus/en-core-web/train.spacy", "dev": "corpus/en-core-web/dev.spacy", "vectors": null, "raw": null, "init_tok2vec": null, "vocab_data": null}, "system": {"gpu_allocator": null, "seed": 0}, "nlp": {"lang": "en", "pipeline": ["tok2vec", "tagger", "parser", "senter", "ner", "attribute_ruler", "lemmatizer"], "disabled": ["senter"], "before_creation": null, "after_creation": null, "after_pipeline_creation": null, "batch_size": 256, "tokenizer": {"@tokenizers": "spacy.Tokenizer.v1"}}, "components": {"tok2vec": {"factory": "tok2vec", "model": {"@architectures": "spacy.Tok2Vec.v1", "embed": {"@architectures": "spacy.MultiHashEmbed.v1", "width": "${components.tok2vec.model.encode:width}", "attrs": ["NORM", "PREFIX", "SUFFIX", "SHAPE"], "rows": [5000, 2500, 2500, 2500], "include_static_vectors": false}, "encode": {"@architectures": "spacy.MaxoutWindowEncoder.v1", "width": 96, "depth": 4, "window_size": 1, "maxout_pieces": 3}}}, "tagger": {"factory": "tagger", "model": {"

In [63]:
json_content = json.loads(config_data)

In [64]:
type(json_content)

dict

In [65]:
json_content["nlp"]["lang"]

'en'

In [66]:
lang_cls = spacy.util.get_lang_class(json_content["nlp"]["lang"])
nlpx = lang_cls.from_config(json_content)
nlpx.from_bytes(data)

In [67]:
doc = nlpx("IBM is an organization")
[(ent.text, ent.label_) for ent in doc.ents]

[('IBM', 'ORG')]

## Predictor and Validator

In [43]:
def predictor(feed_text):
    
    doc = nlpx(feed_text)
    return [(ent.text, ent.label_) for ent in doc.ents]

In [44]:
def validator(x):
    
    return type(x) == str